In [1]:
!pip install mlxtend  
%pip install mlxtend --upgrade 

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import json
from IPython import display
from urllib.request import urlopen
from io import BytesIO
import matplotlib.pyplot as plt
import seaborn as sns
import itertools 

# Allow multiple output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 


In [3]:
import random

# set seed 
my_seed = 609
random.seed(my_seed)
np.random.seed(my_seed)

In [4]:
!git clone https://github.com/adamxrvn/coursework.git

fatal: destination path 'coursework' already exists and is not an empty directory.


In [5]:
!unzip -o coursework/data/dataset_clean.zip
!unzip -o coursework/data/dataset_phones.zip
!unzip -o coursework/data/dataset_delivered.zip

Archive:  coursework/data/dataset_clean.zip
  inflating: content/dataset_clean.csv  
Archive:  coursework/data/dataset_phones.zip
  inflating: content/dataset_phones.csv  
Archive:  coursework/data/dataset_delivered.zip
  inflating: content/dataset_delivered.csv  


In [6]:
# import datasets 
df_delivered = pd.read_csv('content/dataset_delivered.csv',encoding='utf-8')



In [7]:
data = df_delivered.drop(['level_0','index'],axis=1,)
data = data[data['Телефон_new'] != '0']
data = data[data['Группа2'].notna()]
data.reset_index(inplace=True,drop=True)
data.head()

,Дата,НомерЗаказаНаСайте,НовыйСтатус,Регион,Группа2,Группа3,Группа4,Тип,Номенклатура,ТипТовара,...,Телефон_new,ГородМагазина,МагазинЗаказа,ГодДаты,МесяцДаты,ДеньНеделиДаты,ДеньДаты,ВремяОкруглДаты,СуммаСтроки_v,Маржа_v
0,2017-05-01 00:03:00,4513479_TR,Доставлен,Центральный ф.о.,КРУПНОГАБАРИТНЫЙ ТОВАР,ИЗДЕЛИЯ ИЗ ПЛАСТМАССЫ,горшки,КГТ,"ТЕГА, ГОРШОК Веселая Черепаха, Польша",КГТ,...,55574954-50515053535277,0,NaN,2017,5,1,1,00:00:00,152.0,76.00
1,2017-05-01 10:18:00,4513492_TR,К отгрузке,Центральный ф.о.,"ТЕКСТИЛЬ, ТРИКОТАЖ",ДЕТСКАЯ ОДЕЖДА (7-16 лет),жилеты,ОДЕЖДА,"LEADER KIDS, ЖИЛЕТ Тигр, (син), р.140",МГТ,...,55574851-50484949535070,0,NaN,2017,5,1,1,10:00:00,499.0,199.00
2,2017-05-01 15:27:00,4513472_TR,К отгрузке,Центральный ф.о.,"ТЕКСТИЛЬ, ТРИКОТАЖ",ОДЕЖДА ДЛЯ НОВОРОЖДЕННЫХ (0-2 лет),комбинезоны,ОДЕЖДА,"MAKOMA, КОМБИНЕЗОН с капюш. My bear, р. 80, По...",МГТ,...,55575456-53575451535378,0,NaN,2017,5,1,1,15:00:00,678.2,316.77
3,2017-05-01 15:27:00,4513472_TR,К отгрузке,Центральный ф.о.,"ТЕКСТИЛЬ, ТРИКОТАЖ",ДЕТСКИЕ КОЛГОТКИ И НОСКИ,носки,ОДЕЖДА,"ЗАЙКА МОЯ, НОСКИ дет., р. L",МГТ,...,55575456-53575451535378,0,NaN,2017,5,1,1,15:00:00,89.0,51.00
4,2017-05-01 15:27:00,4513472_TR,К отгрузке,Центральный ф.о.,"ТЕКСТИЛЬ, ТРИКОТАЖ",ОДЕЖДА ДЛЯ НОВОРОЖДЕННЫХ (0-2 лет),песочники,ОДЕЖДА,"SOFIJA, ПЕСОЧНИК Bazyl, (сер), р. 80, Польша",МГТ,...,55575456-53575451535378,0,NaN,2017,5,1,1,15:00:00,1080.0,505.20


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374860 entries, 0 to 374859
Data columns (total 30 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Дата                374860 non-null  object 
 1   НомерЗаказаНаСайте  374860 non-null  object 
 2   НовыйСтатус         374860 non-null  object 
 3   Регион              374860 non-null  object 
 4   Группа2             374860 non-null  object 
 5   Группа3             374860 non-null  object 
 6   Группа4             355936 non-null  object 
 7   Тип                 374860 non-null  object 
 8   Номенклатура        374860 non-null  object 
 9   ТипТовара           374860 non-null  object 
 10  Отменено            374860 non-null  object 
 11  Количество          374860 non-null  int64  
 12  Цена                374860 non-null  float64
 13  ЦенаЗакупки         373566 non-null  float64
 14  МесяцДатыЗаказа     374860 non-null  int64  
 15  ГодДатыЗаказа       374860 non-nul

In [9]:
data["Группа2"].value_counts()

ДЕТСКОЕ ПИТАНИЕ              85173
ИГРУШКИ                      71553
ТЕКСТИЛЬ, ТРИКОТАЖ           62947
ПОДГУЗНИКИ                   41360
КОСМЕТИКА/ГИГИЕНА            37542
КРУПНОГАБАРИТНЫЙ ТОВАР       24351
ТОВАРЫ ДЛЯ КОРМЛЕНИЯ         23075
КАНЦТОВАРЫ, КНИГИ, ДИСКИ     12774
ОБУВЬ                         9499
ТОВАРЫ ДЛЯ ЖИВОТНЫХ           4918
СОПУТСТВУЮЩИЕ ТОВАРЫ           801
ЖЕНСКИЕ ШТУЧКИ                 749
ТЕХНИКА И ТОВАРЫ ДЛЯ ДОМА      118
Name: Группа2, dtype: int64

In [10]:
cols = ['Телефон_new']
cols.extend(data['Группа2'].unique())
MB = pd.DataFrame(columns=cols)


MB 


,Телефон_new,КРУПНОГАБАРИТНЫЙ ТОВАР,"ТЕКСТИЛЬ, ТРИКОТАЖ",ОБУВЬ,ИГРУШКИ,"КАНЦТОВАРЫ, КНИГИ, ДИСКИ",ТОВАРЫ ДЛЯ КОРМЛЕНИЯ,СОПУТСТВУЮЩИЕ ТОВАРЫ,КОСМЕТИКА/ГИГИЕНА,ЖЕНСКИЕ ШТУЧКИ,ПОДГУЗНИКИ,ДЕТСКОЕ ПИТАНИЕ,ТЕХНИКА И ТОВАРЫ ДЛЯ ДОМА,ТОВАРЫ ДЛЯ ЖИВОТНЫХ


In [11]:
# pd.pivot_table(data[data['МесяцДаты']==5][data['ДеньДаты']==1][['Телефон_new','Группа2']], index='Телефон_new', columns='Группа2', aggfunc=len, fill_value=0).reset_index()

In [12]:
# Calculate transactions
def get_transactions(data):
  cols = ['Телефон_new']
  cols.extend(data['Группа2'].unique())
  MB = pd.DataFrame(columns=cols)

  range_of_months = data['МесяцДаты'].unique()
  for i in range_of_months:
    range_of_days = data[data['МесяцДаты']==i]['ДеньДаты'].unique()

    for j in range_of_days:
      current_day_pv = pd.pivot_table(data[data['МесяцДаты']==i][data['ДеньДаты']==j][['Телефон_new','Группа2']], index='Телефон_new', columns='Группа2', aggfunc=len, fill_value=0).reset_index()
      MB = pd.concat([MB,current_day_pv]).fillna(0)

  MB.reset_index(drop=True, inplace=True)
  return MB


# Transactions - relative to unique client
def customer_transactions(data):
    return pd.pivot_table(data[['Телефон_new','Группа2']], 
                        index='Телефон_new', columns='Группа2', 
                        aggfunc=len, fill_value=0).reset_index(drop=True)

def transform_mb(MB):
  cols = ['ДЕТСКОЕ ПИТАНИЕ', 'ЖЕНСКИЕ ШТУЧКИ', 'ИГРУШКИ',
       'КАНЦТОВАРЫ, КНИГИ, ДИСКИ', 'КОСМЕТИКА/ГИГИЕНА',
       'КРУПНОГАБАРИТНЫЙ ТОВАР', 'ОБУВЬ', 'ПОДГУЗНИКИ', 'СОПУТСТВУЮЩИЕ ТОВАРЫ',
       'ТЕКСТИЛЬ, ТРИКОТАЖ', 'ТЕХНИКА И ТОВАРЫ ДЛЯ ДОМА',
       'ТОВАРЫ ДЛЯ ЖИВОТНЫХ', 'ТОВАРЫ ДЛЯ КОРМЛЕНИЯ']
  MB['number_of_checks'] = MB[cols].astype(bool).sum(axis=1)
  return MB[MB['number_of_checks'] == 2].drop(['number_of_checks'],axis=1).astype(int).reset_index(drop=True)




In [13]:
MB = get_transactions(data)
MB

,Телефон_new,КРУПНОГАБАРИТНЫЙ ТОВАР,"ТЕКСТИЛЬ, ТРИКОТАЖ",ОБУВЬ,ИГРУШКИ,"КАНЦТОВАРЫ, КНИГИ, ДИСКИ",ТОВАРЫ ДЛЯ КОРМЛЕНИЯ,СОПУТСТВУЮЩИЕ ТОВАРЫ,КОСМЕТИКА/ГИГИЕНА,ЖЕНСКИЕ ШТУЧКИ,ПОДГУЗНИКИ,ДЕТСКОЕ ПИТАНИЕ,ТЕХНИКА И ТОВАРЫ ДЛЯ ДОМА,ТОВАРЫ ДЛЯ ЖИВОТНЫХ
0,55574848-51485152575271,1,0,0,0,0,0,0,0,0,0,0,0.0,0
1,55574848-51535657495078,1,0,0,0,0,0,0,0,0,0,0,0.0,0
2,55574848-53564953564870,0,1,0,0,0,1,0,1,0,0,0,0.0,0
3,55574848-54484949525270,0,0,0,1,0,0,0,0,0,0,0,0.0,0
4,55574848-54505548555577,1,0,0,0,0,0,0,0,0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106617,55575757-54535149574876,0,0,0,0,0,1,0,0,0,0,0,0.0,0
106618,55575757-56484857535475,0,0,0,0,0,0,0,1,0,0,0,0.0,0
106619,55575757-56485550485672,0,0,0,0,0,0,0,0,0,0,0,0.0,2
106620,55575757-56495151555773,1,0,0,0,0,0,0,0,0,0,0,0.0,0


In [14]:
MB.to_csv('MBClient.csv', index=False, encoding='utf-8') 

In [15]:
MB

,Телефон_new,КРУПНОГАБАРИТНЫЙ ТОВАР,"ТЕКСТИЛЬ, ТРИКОТАЖ",ОБУВЬ,ИГРУШКИ,"КАНЦТОВАРЫ, КНИГИ, ДИСКИ",ТОВАРЫ ДЛЯ КОРМЛЕНИЯ,СОПУТСТВУЮЩИЕ ТОВАРЫ,КОСМЕТИКА/ГИГИЕНА,ЖЕНСКИЕ ШТУЧКИ,ПОДГУЗНИКИ,ДЕТСКОЕ ПИТАНИЕ,ТЕХНИКА И ТОВАРЫ ДЛЯ ДОМА,ТОВАРЫ ДЛЯ ЖИВОТНЫХ
0,55574848-51485152575271,1,0,0,0,0,0,0,0,0,0,0,0.0,0
1,55574848-51535657495078,1,0,0,0,0,0,0,0,0,0,0,0.0,0
2,55574848-53564953564870,0,1,0,0,0,1,0,1,0,0,0,0.0,0
3,55574848-54484949525270,0,0,0,1,0,0,0,0,0,0,0,0.0,0
4,55574848-54505548555577,1,0,0,0,0,0,0,0,0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106617,55575757-54535149574876,0,0,0,0,0,1,0,0,0,0,0,0.0,0
106618,55575757-56484857535475,0,0,0,0,0,0,0,1,0,0,0,0.0,0
106619,55575757-56485550485672,0,0,0,0,0,0,0,0,0,0,0,0.0,2
106620,55575757-56495151555773,1,0,0,0,0,0,0,0,0,0,0,0.0,0


In [16]:
MB_ = pd.read_csv('MBClient.csv') 
MB_

,Телефон_new,КРУПНОГАБАРИТНЫЙ ТОВАР,"ТЕКСТИЛЬ, ТРИКОТАЖ",ОБУВЬ,ИГРУШКИ,"КАНЦТОВАРЫ, КНИГИ, ДИСКИ",ТОВАРЫ ДЛЯ КОРМЛЕНИЯ,СОПУТСТВУЮЩИЕ ТОВАРЫ,КОСМЕТИКА/ГИГИЕНА,ЖЕНСКИЕ ШТУЧКИ,ПОДГУЗНИКИ,ДЕТСКОЕ ПИТАНИЕ,ТЕХНИКА И ТОВАРЫ ДЛЯ ДОМА,ТОВАРЫ ДЛЯ ЖИВОТНЫХ
0,55574848-51485152575271,1,0,0,0,0,0,0,0,0,0,0,0.0,0
1,55574848-51535657495078,1,0,0,0,0,0,0,0,0,0,0,0.0,0
2,55574848-53564953564870,0,1,0,0,0,1,0,1,0,0,0,0.0,0
3,55574848-54484949525270,0,0,0,1,0,0,0,0,0,0,0,0.0,0
4,55574848-54505548555577,1,0,0,0,0,0,0,0,0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106617,55575757-54535149574876,0,0,0,0,0,1,0,0,0,0,0,0.0,0
106618,55575757-56484857535475,0,0,0,0,0,0,0,1,0,0,0,0.0,0
106619,55575757-56485550485672,0,0,0,0,0,0,0,0,0,0,0,0.0,2
106620,55575757-56495151555773,1,0,0,0,0,0,0,0,0,0,0,0.0,0


In [17]:
MB = MB_

In [18]:
MB = MB.drop(['Телефон_new'], axis=1)


MB = MB.applymap(lambda x: 1 if x >= 1 else 0)

# columns_to_sum =  ['КРУПНОГАБАРИТНЫЙ ТОВАР', 'ТЕКСТИЛЬ, ТРИКОТАЖ', 'ОБУВЬ',
#        'ИГРУШКИ', 'КАНЦТОВАРЫ, КНИГИ, ДИСКИ', 'ТОВАРЫ ДЛЯ КОРМЛЕНИЯ',
#        'СОПУТСТВУЮЩИЕ ТОВАРЫ', 'КОСМЕТИКА/ГИГИЕНА', 'ЖЕНСКИЕ ШТУЧКИ',
#        'ПОДГУЗНИКИ', 'ДЕТСКОЕ ПИТАНИЕ', 'ТЕХНИКА И ТОВАРЫ ДЛЯ ДОМА',
#        'ТОВАРЫ ДЛЯ ЖИВОТНЫХ']


columns_to_sum = [col for col in MB.columns if col != 'Телефон_new']


MB['sum_cols'] = MB[columns_to_sum].sum(axis=1)

MB = MB[MB['sum_cols'] == 2]

MB = MB.drop(['sum_cols'], axis=1)

MB.reset_index(drop=True,inplace=True)

In [19]:
MB

,КРУПНОГАБАРИТНЫЙ ТОВАР,"ТЕКСТИЛЬ, ТРИКОТАЖ",ОБУВЬ,ИГРУШКИ,"КАНЦТОВАРЫ, КНИГИ, ДИСКИ",ТОВАРЫ ДЛЯ КОРМЛЕНИЯ,СОПУТСТВУЮЩИЕ ТОВАРЫ,КОСМЕТИКА/ГИГИЕНА,ЖЕНСКИЕ ШТУЧКИ,ПОДГУЗНИКИ,ДЕТСКОЕ ПИТАНИЕ,ТЕХНИКА И ТОВАРЫ ДЛЯ ДОМА,ТОВАРЫ ДЛЯ ЖИВОТНЫХ
0,0,1,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,1,1,0,0,0,0,0,0,0,0
2,0,0,0,1,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,1,0,0
4,0,0,0,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19174,0,1,0,0,0,0,0,0,0,1,0,0,0
19175,1,1,0,0,0,0,0,0,0,0,0,0,0
19176,0,0,0,1,0,0,0,1,0,0,0,0,0
19177,0,1,0,1,0,0,0,0,0,0,0,0,0


In [20]:
params = ["support", "confidence", "lift", "expected confidence"] 
res = pd.DataFrame(columns=params) 

In [21]:
cols.remove('Телефон_new')
permutations = {tuple(sorted(p)) for p in itertools.permutations(cols, 2)}
rows = [' + '.join(i) for i in permutations] 
pairs = list(permutations)

In [22]:
import random
print(f"There are {len(pairs)} pairs") 
print(*random.choices(rows,k=5),sep='\n') 

There are 78 pairs
КАНЦТОВАРЫ, КНИГИ, ДИСКИ + ПОДГУЗНИКИ
ОБУВЬ + СОПУТСТВУЮЩИЕ ТОВАРЫ
ЖЕНСКИЕ ШТУЧКИ + ТЕХНИКА И ТОВАРЫ ДЛЯ ДОМА
ОБУВЬ + ПОДГУЗНИКИ
ДЕТСКОЕ ПИТАНИЕ + ИГРУШКИ


In [23]:
temp = pd.DataFrame(0, index=[i for i in rows], columns=params) 

res = res.append(temp, ignore_index=False, sort=False) 
res = res.reset_index() 
res = res.rename(columns={"index": "Pairs"}) 


In [24]:
def support(a, b, df):
    counter = 0
    for i in range(0, len(df)):
        # print(a)
        # print(df)
        if df[a][i] > 0 and df[b][i] > 0:
            counter += 1
    return counter/len(df) 

def confidence(a, b, df):
    counter = 0
    total = 0
    for i in range(0, len(df)):
        if df[a][i] > 0:
            total += 1 
            if df[b][i] > 0:
                counter += 1
    return counter/total 

def expconf(a, b, df):
    counter = 0
    for i in range(0, len(df)):
        if df[b][i] > 0:
            counter += 1
    return counter/len(df) 

In [25]:
for i in range(len(pairs)): 
    sup = support(pairs[i][0], pairs[i][1], MB) 
    conf = confidence(pairs[i][0], pairs[i][1], MB) 
    exconf = expconf(pairs[i][0], pairs[i][1], MB) 
    lift = conf / exconf 
    res['support'][i] = sup 
    res['confidence'][i] = conf 
    res['expected confidence'][i] = exconf 
    res['lift'][i] = lift 

In [26]:
res.sort_values('support',ascending=False)

,Pairs,support,confidence,lift,expected confidence
45,ДЕТСКОЕ ПИТАНИЕ + ПОДГУЗНИКИ,0.145002,0.533167,1.284302,0.415142
43,КОСМЕТИКА/ГИГИЕНА + ПОДГУЗНИКИ,0.096825,0.467405,1.125893,0.415142
8,"ИГРУШКИ + ТЕКСТИЛЬ, ТРИКОТАЖ",0.080557,0.233772,0.871262,0.268314
55,ИГРУШКИ + ПОДГУЗНИКИ,0.061213,0.177637,0.427894,0.415142
4,ИГРУШКИ + КРУПНОГАБАРИТНЫЙ ТОВАР,0.053183,0.154335,1.084246,0.142343
...,...,...,...,...,...
26,ЖЕНСКИЕ ШТУЧКИ + СОПУТСТВУЮЩИЕ ТОВАРЫ,0.0,0.0,0.0,0.009855
62,СОПУТСТВУЮЩИЕ ТОВАРЫ + ТОВАРЫ ДЛЯ ЖИВОТНЫХ,0.0,0.0,0.0,0.018301
59,СОПУТСТВУЮЩИЕ ТОВАРЫ + ТЕХНИКА И ТОВАРЫ ДЛЯ ДОМА,0.0,0.0,0.0,0.000574
56,"ТЕКСТИЛЬ, ТРИКОТАЖ + ТЕХНИКА И ТОВАРЫ ДЛЯ ДОМА",0.0,0.0,0.0,0.000574


In [27]:
res.to_csv('MBClientFinal.csv', index=False, encoding='utf-8') 

# FP-Growth 

In [28]:
from mlxtend.preprocessing import TransactionEncoder 
from mlxtend.frequent_patterns import association_rules, fpgrowth, apriori, fpmax, hmine
warnings.filterwarnings("ignore", category=DeprecationWarning) 


In [29]:
dfp = MB.applymap(lambda x: True if x >= 1 else False)

In [30]:
frequent_itemsets_fp = fpgrowth(dfp, min_support=0.001, use_colnames=True)	


In [31]:
frequent_itemsets_fp.sort_values('support', ascending=False)


,support,itemsets
0,0.415142,(ПОДГУЗНИКИ)
2,0.344596,(ИГРУШКИ)
4,0.271964,(ДЕТСКОЕ ПИТАНИЕ)
1,0.268314,"(ТЕКСТИЛЬ, ТРИКОТАЖ)"
5,0.207154,(КОСМЕТИКА/ГИГИЕНА)
8,0.160592,(ТОВАРЫ ДЛЯ КОРМЛЕНИЯ)
24,0.145002,"(ПОДГУЗНИКИ, ДЕТСКОЕ ПИТАНИЕ)"
6,0.142343,(КРУПНОГАБАРИТНЫЙ ТОВАР)
27,0.096825,"(ПОДГУЗНИКИ, КОСМЕТИКА/ГИГИЕНА)"
7,0.087074,(ОБУВЬ)


# Association rules

In [32]:
rules_ = association_rules(frequent_itemsets_fp, metric="confidence", min_threshold=0.001)

In [33]:
rules_.sort_values('support', ascending=False)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
25,(ДЕТСКОЕ ПИТАНИЕ),(ПОДГУЗНИКИ),0.271964,0.415142,0.145002,0.533167,1.284302,0.032099,1.252822,0.304061
24,(ПОДГУЗНИКИ),(ДЕТСКОЕ ПИТАНИЕ),0.415142,0.271964,0.145002,0.349284,1.284302,0.032099,1.118823,0.378497
31,(КОСМЕТИКА/ГИГИЕНА),(ПОДГУЗНИКИ),0.207154,0.415142,0.096825,0.467405,1.125893,0.010827,1.098130,0.141031
30,(ПОДГУЗНИКИ),(КОСМЕТИКА/ГИГИЕНА),0.415142,0.207154,0.096825,0.233233,1.125893,0.010827,1.034012,0.191185
2,(ИГРУШКИ),"(ТЕКСТИЛЬ, ТРИКОТАЖ)",0.344596,0.268314,0.080557,0.233772,0.871262,-0.011903,0.954919,-0.183972
...,...,...,...,...,...,...,...,...,...,...
87,(СОПУТСТВУЮЩИЕ ТОВАРЫ),"(ТЕКСТИЛЬ, ТРИКОТАЖ)",0.009855,0.268314,0.001825,0.185185,0.690180,-0.000819,0.897978,-0.311942
10,"(КАНЦТОВАРЫ, КНИГИ, ДИСКИ)",(ТОВАРЫ ДЛЯ КОРМЛЕНИЯ),0.070859,0.160592,0.001408,0.019868,0.123714,-0.009972,0.856423,-0.884035
11,(ТОВАРЫ ДЛЯ КОРМЛЕНИЯ),"(КАНЦТОВАРЫ, КНИГИ, ДИСКИ)",0.160592,0.070859,0.001408,0.008766,0.123714,-0.009972,0.937358,-0.894048
82,(ТОВАРЫ ДЛЯ ЖИВОТНЫХ),(КРУПНОГАБАРИТНЫЙ ТОВАР),0.018301,0.142343,0.001147,0.062678,0.440331,-0.001458,0.915008,-0.564216


# Apriori

In [34]:
apriori_res = apriori(dfp, min_support=0.001, use_colnames=True).sort_values("support",ascending=False)

In [35]:
apriori_res.sort_values('support', ascending=False)


,support,itemsets
9,0.415142,(ПОДГУЗНИКИ)
3,0.344596,(ИГРУШКИ)
10,0.271964,(ДЕТСКОЕ ПИТАНИЕ)
1,0.268314,"(ТЕКСТИЛЬ, ТРИКОТАЖ)"
7,0.207154,(КОСМЕТИКА/ГИГИЕНА)
5,0.160592,(ТОВАРЫ ДЛЯ КОРМЛЕНИЯ)
53,0.145002,"(ПОДГУЗНИКИ, ДЕТСКОЕ ПИТАНИЕ)"
0,0.142343,(КРУПНОГАБАРИТНЫЙ ТОВАР)
50,0.096825,"(ПОДГУЗНИКИ, КОСМЕТИКА/ГИГИЕНА)"
2,0.087074,(ОБУВЬ)


# FPMax

In [36]:
fpmax_res = fpmax(dfp, min_support=0.001, use_colnames=True)

In [37]:
fpmax_res.sort_values('support', ascending=False)


,support,itemsets
43,0.145002,"(ПОДГУЗНИКИ, ДЕТСКОЕ ПИТАНИЕ)"
38,0.096825,"(ПОДГУЗНИКИ, КОСМЕТИКА/ГИГИЕНА)"
41,0.080557,"(ИГРУШКИ, ТЕКСТИЛЬ, ТРИКОТАЖ)"
44,0.061213,"(ПОДГУЗНИКИ, ИГРУШКИ)"
29,0.053183,"(ИГРУШКИ, КРУПНОГАБАРИТНЫЙ ТОВАР)"
16,0.046196,"(ИГРУШКИ, КАНЦТОВАРЫ, КНИГИ, ДИСКИ)"
23,0.044632,"(ТЕКСТИЛЬ, ТРИКОТАЖ, ОБУВЬ)"
40,0.039210,"(ПОДГУЗНИКИ, ТЕКСТИЛЬ, ТРИКОТАЖ)"
34,0.037332,"(ПОДГУЗНИКИ, ТОВАРЫ ДЛЯ КОРМЛЕНИЯ)"
33,0.030763,"(ТОВАРЫ ДЛЯ КОРМЛЕНИЯ, ДЕТСКОЕ ПИТАНИЕ)"


# Hmine

In [38]:
hmine_res = hmine(dfp, min_support=0.001,use_colnames=True,verbose=False)


In [39]:
hmine_res.sort_values('support', ascending=False)


,support,itemsets
50,0.415142,(ПОДГУЗНИКИ)
27,0.344596,(ИГРУШКИ)
53,0.271964,(ДЕТСКОЕ ПИТАНИЕ)
10,0.268314,"(ТЕКСТИЛЬ, ТРИКОТАЖ)"
45,0.207154,(КОСМЕТИКА/ГИГИЕНА)
40,0.160592,(ТОВАРЫ ДЛЯ КОРМЛЕНИЯ)
51,0.145002,"(ПОДГУЗНИКИ, ДЕТСКОЕ ПИТАНИЕ)"
0,0.142343,(КРУПНОГАБАРИТНЫЙ ТОВАР)
46,0.096825,"(ПОДГУЗНИКИ, КОСМЕТИКА/ГИГИЕНА)"
20,0.087074,(ОБУВЬ)


# MBA based on Clusters

In [40]:
def process_clusters(df, num_of_clusters):
  results = []
  
  
  
  for cluster in range(num_of_clusters):
    local_resuts = {
                "FP-Growth":None,
                "Association rules":None,
                "Apriori":None,
                "FPMax":None,
                "Hmine":None}
    data = df

    data = data[data['Labels'] == cluster]

    cols = ['Телефон_new']
    cols.extend(data['Группа2'].unique())
    MB = pd.DataFrame(columns=cols)

    #
    MB = get_transactions(data)
    MB_ = MB

    MB = MB.drop(['Телефон_new'], axis=1)


    MB = MB.applymap(lambda x: 1 if x >= 1 else 0)

    # columns_to_sum =  ['КРУПНОГАБАРИТНЫЙ ТОВАР', 'ТЕКСТИЛЬ, ТРИКОТАЖ', 'ОБУВЬ',
    #        'ИГРУШКИ', 'КАНЦТОВАРЫ, КНИГИ, ДИСКИ', 'ТОВАРЫ ДЛЯ КОРМЛЕНИЯ',
    #        'СОПУТСТВУЮЩИЕ ТОВАРЫ', 'КОСМЕТИКА/ГИГИЕНА', 'ЖЕНСКИЕ ШТУЧКИ',
    #        'ПОДГУЗНИКИ', 'ДЕТСКОЕ ПИТАНИЕ', 'ТЕХНИКА И ТОВАРЫ ДЛЯ ДОМА',
    #        'ТОВАРЫ ДЛЯ ЖИВОТНЫХ']


    columns_to_sum = [col for col in MB.columns if col != 'Телефон_new']


    MB['sum_cols'] = MB[columns_to_sum].sum(axis=1)

    MB = MB[MB['sum_cols'] == 2]

    MB = MB.drop(['sum_cols'], axis=1)

    MB.reset_index(drop=True,inplace=True)

    #


  

    # FP-Growth
    dfp = MB.drop(['Телефон_new'], axis=1, errors='ignore').applymap(lambda x: True if x >= 1 else False)
    frequent_itemsets_fp = fpgrowth(dfp, min_support=0.001, use_colnames=True)	
    
    local_resuts['FP-Growth'] = frequent_itemsets_fp
    
    # association rules
    rules_fp = association_rules(frequent_itemsets_fp, metric="confidence", min_threshold=0.02)

    local_resuts['Association rules'] = rules_fp

    #Apriori
    apriori_res = apriori(dfp, min_support=0.001, use_colnames=True).sort_values("support",ascending=False)

    local_resuts['Apriori'] = apriori_res

    # fpmax
    fpmax_res = fpmax(dfp, min_support=0.001, use_colnames=True)

    local_resuts['FPMax'] = fpmax_res

    #hmine
    hmine_res = hmine(dfp, min_support=0.001,use_colnames=True,verbose=False)

    local_resuts['Hmine'] = hmine_res

    results.append(local_resuts)

  return results

In [41]:
df_c = pd.read_csv('df_12_clusters_km.csv')

In [42]:
df_c['Labels'].dropna(inplace=True)

In [43]:
df_c.head()

,Телефон_new,Выручка_bought,Маржа_bought,Средний чек_bought,Кол-во чеков_bought,Среднее кол-во товаров в чеке_bought,Центральный ф.о.,Приволжский ф.о.,Северо-Западный ф.о.,Южный ф.о.,...,target,Ценность клиента_bought,Кол-во уникальных клиентов_bought,Средняя маржа_bought,Абсолютная маржа_bought,Кол-во товаров_bought,КГТ выручка,ИНОЕ выручка,ИГРУШКИт выручка,ОДЕЖДА выручка
0,55574954-50515053535277,992,0.44,318,3,2,1.0,0.0,0.0,0.0,...,1,435,1,145.00,435,3,327.36,0.0,327.36,327.36
1,55574851-50484949535070,1698,0.26,849,2,2,1.0,0.0,0.0,0.0,...,0,444,1,222.00,444,2,0.00,0.0,0.00,1698.00
2,55574954-56545048505170,4038,0.28,281,18,9,1.0,0.0,0.0,0.0,...,0,1132,1,62.89,1132,19,242.28,0.0,2907.36,0.00
3,55574954-53555555545178,1280,0.39,427,2,3,1.0,0.0,0.0,0.0,...,0,501,1,250.50,501,3,0.00,0.0,0.00,1280.00
4,55574857-54544951504973,6381,0.14,456,14,14,1.0,0.0,0.0,0.0,...,0,921,1,65.79,921,17,446.67,0.0,1340.01,893.34


In [44]:
df_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79389 entries, 0 to 79388
Data columns (total 46 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Телефон_new                            79389 non-null  object 
 1   Выручка_bought                         79389 non-null  int64  
 2   Маржа_bought                           79389 non-null  float64
 3   Средний чек_bought                     79389 non-null  int64  
 4   Кол-во чеков_bought                    79389 non-null  int64  
 5   Среднее кол-во товаров в  чеке_bought  79389 non-null  int64  
 6   Центральный ф.о.                       79389 non-null  float64
 7   Приволжский ф.о.                       79389 non-null  float64
 8   Северо-Западный ф.о.                   79389 non-null  float64
 9   Южный ф.о.                             79389 non-null  float64
 10  Сибирский ф.о.                         79389 non-null  float64
 11  Ур

In [45]:
phone_cluster = df_c[['Телефон_new','Labels']].drop_duplicates()


In [46]:
phone_cluster

,Телефон_new,Labels
0,55574954-50515053535277,9
1,55574851-50484949535070,4
2,55574954-56545048505170,9
3,55574954-53555555545178,4
4,55574857-54544951504973,3
...,...,...
79384,55574949-57565751545277,9
79385,55574854-55505648555577,1
79386,55575053-50544856505076,4
79387,55575054-55574849485475,9


In [132]:
phone_cluster.reset_index(inplace=True,drop=True)


In [48]:
data_c = df_delivered.drop(['level_0','index'],axis=1,)
data_c = data_c[data_c['Телефон_new'] != '0']
data_c = data_c[data_c['Группа2'].notna()]
data_c.reset_index(inplace=True,drop=True)

data_c = data_c.merge(df_c[['Телефон_new','Labels']], on='Телефон_new')




In [49]:
data_c.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 322790 entries, 0 to 322789
Data columns (total 31 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Дата                322790 non-null  object 
 1   НомерЗаказаНаСайте  322790 non-null  object 
 2   НовыйСтатус         322790 non-null  object 
 3   Регион              322790 non-null  object 
 4   Группа2             322790 non-null  object 
 5   Группа3             322790 non-null  object 
 6   Группа4             306291 non-null  object 
 7   Тип                 322790 non-null  object 
 8   Номенклатура        322790 non-null  object 
 9   ТипТовара           322790 non-null  object 
 10  Отменено            322790 non-null  object 
 11  Количество          322790 non-null  int64  
 12  Цена                322790 non-null  float64
 13  ЦенаЗакупки         321882 non-null  float64
 14  МесяцДатыЗаказа     322790 non-null  int64  
 15  ГодДатыЗаказа       322790 non-nul

In [50]:
all_clusters = process_clusters(data_c, len(df_c.Labels.unique()))

In [51]:
all_clusters[3]['Apriori']

,support,itemsets
5,0.652904,(ДЕТСКОЕ ПИТАНИЕ)
4,0.592722,(ПОДГУЗНИКИ)
36,0.381386,"(ПОДГУЗНИКИ, ДЕТСКОЕ ПИТАНИЕ)"
3,0.289713,(КОСМЕТИКА/ГИГИЕНА)
6,0.151155,(ТОВАРЫ ДЛЯ КОРМЛЕНИЯ)
31,0.129811,"(ПОДГУЗНИКИ, КОСМЕТИКА/ГИГИЕНА)"
2,0.127362,(ИГРУШКИ)
7,0.088174,"(ТЕКСТИЛЬ, ТРИКОТАЖ)"
32,0.084325,"(ДЕТСКОЕ ПИТАНИЕ, КОСМЕТИКА/ГИГИЕНА)"
40,0.073478,"(ТОВАРЫ ДЛЯ КОРМЛЕНИЯ, ДЕТСКОЕ ПИТАНИЕ)"


In [52]:
import pickle


def cluster_mba_save(all_clusters):
    with open("all_clusters.pickle", 'wb') as f:
        pickle.dump(all_clusters, f, pickle.HIGHEST_PROTOCOL)

def cluster_mba_load(path):
    with open('path', 'rb') as f:
        return pickle.load(f)

cluster_mba_save(all_clusters)

# Recommendation systems

In [53]:
!pip install scikit-surprise

In [54]:
# SVD

In [55]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold


In [96]:
from collections import defaultdict
# Official function. 
# Reference: https://surprise.readthedocs.io/en/stable/FAQ.html?highlight=top#how-to-get-the-top-n-recommendations-for-each-user

def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [57]:
data

,Дата,НомерЗаказаНаСайте,НовыйСтатус,Регион,Группа2,Группа3,Группа4,Тип,Номенклатура,ТипТовара,...,Телефон_new,ГородМагазина,МагазинЗаказа,ГодДаты,МесяцДаты,ДеньНеделиДаты,ДеньДаты,ВремяОкруглДаты,СуммаСтроки_v,Маржа_v
0,2017-05-01 00:03:00,4513479_TR,Доставлен,Центральный ф.о.,КРУПНОГАБАРИТНЫЙ ТОВАР,ИЗДЕЛИЯ ИЗ ПЛАСТМАССЫ,горшки,КГТ,"ТЕГА, ГОРШОК Веселая Черепаха, Польша",КГТ,...,55574954-50515053535277,0,NaN,2017,5,1,1,00:00:00,152.0,76.00
1,2017-05-01 10:18:00,4513492_TR,К отгрузке,Центральный ф.о.,"ТЕКСТИЛЬ, ТРИКОТАЖ",ДЕТСКАЯ ОДЕЖДА (7-16 лет),жилеты,ОДЕЖДА,"LEADER KIDS, ЖИЛЕТ Тигр, (син), р.140",МГТ,...,55574851-50484949535070,0,NaN,2017,5,1,1,10:00:00,499.0,199.00
2,2017-05-01 15:27:00,4513472_TR,К отгрузке,Центральный ф.о.,"ТЕКСТИЛЬ, ТРИКОТАЖ",ОДЕЖДА ДЛЯ НОВОРОЖДЕННЫХ (0-2 лет),комбинезоны,ОДЕЖДА,"MAKOMA, КОМБИНЕЗОН с капюш. My bear, р. 80, По...",МГТ,...,55575456-53575451535378,0,NaN,2017,5,1,1,15:00:00,678.2,316.77
3,2017-05-01 15:27:00,4513472_TR,К отгрузке,Центральный ф.о.,"ТЕКСТИЛЬ, ТРИКОТАЖ",ДЕТСКИЕ КОЛГОТКИ И НОСКИ,носки,ОДЕЖДА,"ЗАЙКА МОЯ, НОСКИ дет., р. L",МГТ,...,55575456-53575451535378,0,NaN,2017,5,1,1,15:00:00,89.0,51.00
4,2017-05-01 15:27:00,4513472_TR,К отгрузке,Центральный ф.о.,"ТЕКСТИЛЬ, ТРИКОТАЖ",ОДЕЖДА ДЛЯ НОВОРОЖДЕННЫХ (0-2 лет),песочники,ОДЕЖДА,"SOFIJA, ПЕСОЧНИК Bazyl, (сер), р. 80, Польша",МГТ,...,55575456-53575451535378,0,NaN,2017,5,1,1,15:00:00,1080.0,505.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374855,2017-06-30 22:55:00,5031477_TR,Доставлен,Центральный ф.о.,ДЕТСКОЕ ПИТАНИЕ,ПЮРЕ,пюре овощные,ППКП,"СЭМПЕР, ПЮРЕ цукини, с 4-х мес., (125 г)",МГТ,...,55574857-57485151544876,0,NaN,2017,6,5,30,22:00:00,490.0,77.28
374856,2017-06-30 22:55:00,5031477_TR,Доставлен,Центральный ф.о.,ДЕТСКОЕ ПИТАНИЕ,ПЮРЕ,пюре мясные,ППКП,"СЭМПЕР, ПЮРЕ телятина, (мономясо), с 6 мес., (...",МГТ,...,55574857-57485151544876,0,NaN,2017,6,5,30,22:00:00,198.0,1.00
374857,2017-06-30 23:13:00,5031575_TR,К отгрузке,Северо-Западный ф.о.,"ТЕКСТИЛЬ, ТРИКОТАЖ",ДЕТСКАЯ ОДЕЖДА (7-16 лет),футболки,ОДЕЖДА,"BEMBI, ФУТБОЛКА (бел), р.128",МГТ,...,55575348-48504953555074,0,NaN,2017,6,5,30,23:00:00,199.0,103.00
374858,2017-06-30 23:58:00,5031783_TR,Доставлен,Приволжский ф.о.,ДЕТСКОЕ ПИТАНИЕ,ПЮРЕ,пюре ассорти,ППКП,"ХИПП, ПЮРЕ каша груша-зерновые хлопья, с 4 мес...",МГТ,...,55575450-53495456535575,0,NaN,2017,6,5,30,23:00:00,336.0,0.40


In [58]:
ones = pd.DataFrame(data.shape[0]*[1],columns=['present'])
data_rs = data[['Телефон_new','Группа4']]
data_rs = pd.concat([data_rs, ones],axis=1)
data_rs

,Телефон_new,Группа4,present
0,55574954-50515053535277,горшки,1
1,55574851-50484949535070,жилеты,1
2,55575456-53575451535378,комбинезоны,1
3,55575456-53575451535378,носки,1
4,55575456-53575451535378,песочники,1
...,...,...,...
374855,55574857-57485151544876,пюре овощные,1
374856,55574857-57485151544876,пюре мясные,1
374857,55575348-48504953555074,футболки,1
374858,55575450-53495456535575,пюре ассорти,1


In [59]:
# # Create a reader specifying the rating scale
reader = Reader(rating_scale=(0, 1))  # Assuming binary presence of goods in the bill

# # Load the dataset into Surprise
dataset = Dataset.load_from_df(data_rs, reader)

In [61]:
# kf = KFold(n_splits=5)
# algo = SVD()

# for trainset, testset in kf.split(dataset):
#     algo.fit(trainset)
#     predictions = algo.test(testset)
#     precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)

#     # Precision and recall can then be averaged over all users
#     print(sum(prec for prec in precisions.values()) / len(precisions))
#     print(sum(rec for rec in recalls.values()) / len(recalls))

In [62]:
# # NMF
# from surprise import NMF


In [69]:
!pip install tabulate

In [70]:
from surprise.model_selection import cross_validate, KFold
from tabulate import tabulate
import datetime
import time
from surprise import (
    BaselineOnly,
    CoClustering,
    Dataset,
    KNNBaseline,
    KNNBasic,
    KNNWithMeans,
    NMF,
    NormalPredictor,
    SlopeOne,
    SVD,
    SVDpp,
)

In [71]:
kf = KFold(random_state=609)  # folds will be the same for all algorithms.


In [95]:

# The algorithms to cross-validate
algos = (
    SVD(random_state=609),
    SVDpp(random_state=609, cache_ratings=False),
    SVDpp(random_state=609, cache_ratings=True),
    NMF(random_state=609),

    # KNNBasic(),
#    KNNWithMeans(),
#    KNNBaseline(),
#    CoClustering(random_state=609),
#    SlopeOne(),
#    BaselineOnly(),
#    NormalPredictor(),
 
)

algo_results = pd.DataFrame(columns=["Name","RMSE","MAE","TIME"])
algo_results = algo_results.set_index("Name")



for algo in algos:
    start = time.time()
    out = cross_validate(algo, dataset, ["rmse", "mae"], kf)
    cv_time = str(datetime.timedelta(seconds=int(time.time() - start)))
    link = algo.__class__.__name__
    cache_ratings=False
    try: 
        cache_ratings = algo.__getattribute__("cache_ratings")
    except:
        cache_ratings=False

    if cache_ratings:
        link += " w/ cache_ratings"
    mean_rmse = "{:.3f}".format(np.mean(out["test_rmse"]))
    mean_mae = "{:.3f}".format(np.mean(out["test_mae"]))
    new_line = {"Name" :link, "RMSE":mean_rmse, "MAE": mean_mae,  "TIME": cv_time}
    print(tabulate([new_line], tablefmt="pipe"))  # print current algo perf
    algo_results = algo_results.append(new_line, ignore_index=True)


algo_results.sort_values('RMSE',ascending=True)

|:----|------:|------:|:--------|
| SVD | 0.011 | 0.003 | 0:00:13 |
|:------|------:|------:|:--------|
| SVDpp | 0.004 | 0.001 | 0:00:24 |
|:-----------------------|------:|------:|:--------|
| SVDpp w/ cache_ratings | 0.004 | 0.001 | 0:00:23 |
|:----|------:|------:|:--------|
| NMF | 0.062 | 0.057 | 0:00:24 |


,RMSE,MAE,TIME,Name
1,0.004,0.001,0:00:24,SVDpp
2,0.004,0.001,0:00:23,SVDpp w/ cache_ratings
0,0.011,0.003,0:00:13,SVD
3,0.062,0.057,0:00:24,NMF


In [179]:
algo_results.sort_values('RMSE',ascending=True)

,RMSE,MAE,TIME,Name
1,0.004,0.001,0:00:24,SVDpp
2,0.004,0.001,0:00:23,SVDpp w/ cache_ratings
0,0.011,0.003,0:00:13,SVD
3,0.062,0.057,0:00:24,NMF


In [103]:

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
trainset = dataset.build_full_trainset()

algo = SVD()
algo.fit(trainset)

testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

# # Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])
    break

55574954-50515053535277 ['комбинезоны', 'носки', 'песочники', 'боди', 'комплекты, наборы', 'матрасы в кровать', 'пляжная обувь', 'качели', 'кровати', 'куклы/пупсы']


# Now run on each cluster

In [104]:
data_c

,Дата,НомерЗаказаНаСайте,НовыйСтатус,Регион,Группа2,Группа3,Группа4,Тип,Номенклатура,ТипТовара,...,ГородМагазина,МагазинЗаказа,ГодДаты,МесяцДаты,ДеньНеделиДаты,ДеньДаты,ВремяОкруглДаты,СуммаСтроки_v,Маржа_v,Labels
0,2017-05-01 00:03:00,4513479_TR,Доставлен,Центральный ф.о.,КРУПНОГАБАРИТНЫЙ ТОВАР,ИЗДЕЛИЯ ИЗ ПЛАСТМАССЫ,горшки,КГТ,"ТЕГА, ГОРШОК Веселая Черепаха, Польша",КГТ,...,0,NaN,2017,5,1,1,00:00:00,152.0,76.00,9
1,2017-05-01 00:03:00,4513479_TR,Доставлен,Центральный ф.о.,ОБУВЬ,ОБУВЬ ДЕТСКАЯ,резиновая обувь,ОДЕЖДА,"ДЮНА, РЕЗИНА неутепленная, (сер), р. 23-26",МГТ,...,0,NaN,2017,5,1,1,00:00:00,560.0,245.00,9
2,2017-06-18 22:15:00,4930289_TR,Доставлен,Центральный ф.о.,ИГРУШКИ,ИГРУШКИ ДЛЯ РАЗВИТИЯ МАЛЫШЕЙ,прочие игрушки для развития,ИГРУШКИ,"АЗБУКВАРИК, СМАРТФОН Песенки В.Шаинского, (в асс)",МГТ,...,0,NaN,2017,6,7,18,22:00:00,280.0,113.68,9
3,2017-05-01 10:18:00,4513492_TR,К отгрузке,Центральный ф.о.,"ТЕКСТИЛЬ, ТРИКОТАЖ",ДЕТСКАЯ ОДЕЖДА (7-16 лет),жилеты,ОДЕЖДА,"LEADER KIDS, ЖИЛЕТ Тигр, (син), р.140",МГТ,...,0,NaN,2017,5,1,1,10:00:00,499.0,199.00,4
4,2017-05-01 10:18:00,4513492_TR,К отгрузке,Центральный ф.о.,"ТЕКСТИЛЬ, ТРИКОТАЖ",ДЕТСКАЯ ОДЕЖДА (7-16 лет),джемпера,ОДЕЖДА,"S`COOL, ТОЛСТОВКА, (сер.мел+черн+бирюза), р.140",МГТ,...,0,NaN,2017,5,1,1,10:00:00,1199.0,245.00,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322785,2017-06-30 20:21:00,5030177_TR,Доставлен,Центральный ф.о.,ОБУВЬ,ОБУВЬ ДЕТСКАЯ,текстильная обувь,ОДЕЖДА,"КОТОФЕЙ, ТУФЛИ (син), р. 20-26",МГТ,...,0,NaN,2017,6,5,30,20:00:00,740.0,305.00,4
322786,2017-06-30 20:21:00,5030177_TR,Доставлен,Центральный ф.о.,ОБУВЬ,ОБУВЬ ДЕТСКАЯ,текстильная обувь,ОДЕЖДА,"КОТОФЕЙ, ТУФЛИ (сер), р. 19-22",МГТ,...,0,NaN,2017,6,5,30,20:00:00,870.0,435.00,4
322787,2017-06-30 20:21:00,5030177_TR,Доставлен,Центральный ф.о.,ОБУВЬ,ОБУВЬ ДЕТСКАЯ,текстильная обувь,ОДЕЖДА,"КОТОФЕЙ, ТУФЛИ (син+бел), р. 19-22",МГТ,...,0,NaN,2017,6,5,30,20:00:00,870.0,435.00,4
322788,2017-06-30 20:21:00,5030297_TR,Доставлен,Центральный ф.о.,ИГРУШКИ,ИГРУШКИ ДЛЯ РАЗВИТИЯ МАЛЫШЕЙ,игрушки из дерева,ИГРУШКИ,"МИР ДЕРЕВЯННЫХ ИГРУШЕК, ДЕРГУНЧИК Нюша",МГТ,...,0,NaN,2017,6,5,30,20:00:00,225.0,85.50,9


In [183]:
def recommender_systems_by_cluster(df, number_of_cluster, verbose = True):
    results = []
    for cluster in range(number_of_cluster):
        print('|'*6, '\tCluster', cluster, '|'*6)
        # Prepare Data
        # only selected cluster
        data = df[df['Labels']==cluster].reset_index(drop=True)
        ones = pd.DataFrame(data.shape[0]*[1],columns=['present'])
        data_rs = data[['Телефон_new','Группа4']]
        data_rs = pd.concat([data_rs, ones],axis=1)
        # print(data_rs)


        # # Create a reader specifying the rating scale
        reader = Reader(rating_scale=(0, 1))  # Assuming binary presence of goods in the bill

        # # Load the dataset into Surprise
        dataset = Dataset.load_from_df(data_rs, reader)
        
        # save metrics
        algo_results = pd.DataFrame(columns=["Name","RMSE","MAE","TIME"])
        algo_results = algo_results.set_index("Name")

        # The algorithms to cross-validate
        algos = (
            SVD(random_state=609),
            SVDpp(random_state=609, cache_ratings=False),
            SVDpp(random_state=609, cache_ratings=True),
            NMF(random_state=609),

            # KNNBasic(),
        #    KNNWithMeans(),
        #    KNNBaseline(),
        #    CoClustering(random_state=609),
        #    SlopeOne(),
        #    BaselineOnly(),
        #    NormalPredictor(),
        
        )


        for algo in algos:
            start = time.time()
            out = cross_validate(algo, dataset, ["rmse", "mae"], kf)
            cv_time = str(datetime.timedelta(seconds=int(time.time() - start)))
            link = algo.__class__.__name__
            cache_ratings=False
            try: 
                cache_ratings = algo.__getattribute__("cache_ratings")
            except:
                cache_ratings=False

            if cache_ratings:
                link += " w/ cache_ratings"
            mean_rmse = "{:.3f}".format(np.mean(out["test_rmse"]))
            mean_mae = "{:.3f}".format(np.mean(out["test_mae"]))
            new_line = {"Name" :link, "RMSE":mean_rmse, "MAE": mean_mae,  "TIME": cv_time}
            print(tabulate([new_line], tablefmt="pipe"))  # print current algo perf
            algo_results = algo_results.append(new_line, ignore_index=True)

        results.append(algo_results)
        print('\n')
    
    return results
        

In [184]:
output = recommender_systems_by_cluster(data_c, len(df_c.Labels.unique()))

|||||| 	Cluster 0 ||||||
|:----|------:|-----:|:--------|
| SVD | 0.042 | 0.02 | 0:00:00 |
|:------|-----:|-----:|:--------|
| SVDpp | 0.02 | 0.01 | 0:00:00 |
|:-----------------------|-----:|-----:|:--------|
| SVDpp w/ cache_ratings | 0.02 | 0.01 | 0:00:00 |
|:----|------:|------:|:--------|
| NMF | 0.066 | 0.057 | 0:00:23 |


|||||| 	Cluster 1 ||||||
|:----|------:|------:|:--------|
| SVD | 0.024 | 0.008 | 0:00:02 |
|:------|------:|------:|:--------|
| SVDpp | 0.011 | 0.003 | 0:00:02 |
|:-----------------------|------:|------:|:--------|
| SVDpp w/ cache_ratings | 0.011 | 0.003 | 0:00:02 |
|:----|------:|------:|:--------|
| NMF | 0.064 | 0.059 | 0:00:04 |


|||||| 	Cluster 2 ||||||
|:----|-----:|------:|:--------|
| SVD | 0.02 | 0.006 | 0:00:00 |
|:------|------:|------:|:--------|
| SVDpp | 0.014 | 0.005 | 0:00:00 |
|:-----------------------|------:|------:|:--------|
| SVDpp w/ cache_ratings | 0.014 | 0.005 | 0:00:00 |
|:----|------:|------:|:--------|
| NMF | 0.055 | 0.026 | 0

# Tables for the report

In [208]:
output[0][['RMSE','Name']].set_index('Name')

,RMSE
Name,
SVD,0.042
SVDpp,0.020
SVDpp w/ cache_ratings,0.020
NMF,0.066


In [214]:
temp = pd.DataFrame()

for i in output:
    temp = pd.concat([temp,i[['RMSE','Name']].set_index('Name')],axis=1)

In [216]:
temp.T

Name,SVD,SVDpp,SVDpp w/ cache_ratings,NMF
RMSE,0.042,0.020,0.020,0.066
RMSE,0.024,0.011,0.011,0.064
RMSE,0.020,0.014,0.014,0.055
RMSE,0.023,0.009,0.009,0.061
RMSE,0.026,0.011,0.011,0.065
RMSE,0.028,0.014,0.014,0.063
RMSE,0.044,0.031,0.031,0.073
RMSE,0.039,0.020,0.020,0.066
RMSE,0.039,0.019,0.019,0.091
RMSE,0.024,0.010,0.010,0.064


In [174]:
group0 = 3

all_clusters[group0]['Apriori'].sort_values('support',ascending=False).reset_index(drop=True)[0:10]

,support,itemsets
0,0.652904,(ДЕТСКОЕ ПИТАНИЕ)
1,0.592722,(ПОДГУЗНИКИ)
2,0.381386,"(ПОДГУЗНИКИ, ДЕТСКОЕ ПИТАНИЕ)"
3,0.289713,(КОСМЕТИКА/ГИГИЕНА)
4,0.151155,(ТОВАРЫ ДЛЯ КОРМЛЕНИЯ)
5,0.129811,"(ПОДГУЗНИКИ, КОСМЕТИКА/ГИГИЕНА)"
6,0.127362,(ИГРУШКИ)
7,0.088174,"(ТЕКСТИЛЬ, ТРИКОТАЖ)"
8,0.084325,"(ДЕТСКОЕ ПИТАНИЕ, КОСМЕТИКА/ГИГИЕНА)"
9,0.073478,"(ТОВАРЫ ДЛЯ КОРМЛЕНИЯ, ДЕТСКОЕ ПИТАНИЕ)"


In [175]:
all_clusters[group0]['FP-Growth'].sort_values('support',ascending=False).reset_index(drop=True)[0:10]

,support,itemsets
0,0.652904,(ДЕТСКОЕ ПИТАНИЕ)
1,0.592722,(ПОДГУЗНИКИ)
2,0.381386,"(ПОДГУЗНИКИ, ДЕТСКОЕ ПИТАНИЕ)"
3,0.289713,(КОСМЕТИКА/ГИГИЕНА)
4,0.151155,(ТОВАРЫ ДЛЯ КОРМЛЕНИЯ)
5,0.129811,"(ПОДГУЗНИКИ, КОСМЕТИКА/ГИГИЕНА)"
6,0.127362,(ИГРУШКИ)
7,0.088174,"(ТЕКСТИЛЬ, ТРИКОТАЖ)"
8,0.084325,"(ДЕТСКОЕ ПИТАНИЕ, КОСМЕТИКА/ГИГИЕНА)"
9,0.073478,"(ТОВАРЫ ДЛЯ КОРМЛЕНИЯ, ДЕТСКОЕ ПИТАНИЕ)"


In [170]:
all_clusters[group0]['Association rules'].reset_index(drop=True)[0:10]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(ПОДГУЗНИКИ),(ДЕТСКОЕ ПИТАНИЕ),0.592722,0.652904,0.381386,0.643447,0.985516,-0.005605,0.973477,-0.034829
1,(ДЕТСКОЕ ПИТАНИЕ),(ПОДГУЗНИКИ),0.652904,0.592722,0.381386,0.584137,0.985516,-0.005605,0.979356,-0.040622
2,(ПОДГУЗНИКИ),"(ТЕКСТИЛЬ, ТРИКОТАЖ)",0.592722,0.088174,0.013296,0.022432,0.254409,-0.038966,0.932750,-0.877986
3,"(ТЕКСТИЛЬ, ТРИКОТАЖ)",(ПОДГУЗНИКИ),0.088174,0.592722,0.013296,0.150794,0.254409,-0.038966,0.479598,-0.762700
4,"(ТЕКСТИЛЬ, ТРИКОТАЖ)",(КОСМЕТИКА/ГИГИЕНА),0.088174,0.289713,0.012946,0.146825,0.506796,-0.012599,0.832522,-0.516274
5,(КОСМЕТИКА/ГИГИЕНА),"(ТЕКСТИЛЬ, ТРИКОТАЖ)",0.289713,0.088174,0.012946,0.044686,0.506796,-0.012599,0.954478,-0.578081
6,(ИГРУШКИ),"(ТЕКСТИЛЬ, ТРИКОТАЖ)",0.127362,0.088174,0.011896,0.093407,1.059349,0.000666,1.005772,0.064201
7,"(ТЕКСТИЛЬ, ТРИКОТАЖ)",(ИГРУШКИ),0.088174,0.127362,0.011896,0.134921,1.059349,0.000666,1.008738,0.061442
8,"(ТЕКСТИЛЬ, ТРИКОТАЖ)",(ДЕТСКОЕ ПИТАНИЕ),0.088174,0.652904,0.034640,0.392857,0.601707,-0.022929,0.571687,-0.420608
9,(ДЕТСКОЕ ПИТАНИЕ),"(ТЕКСТИЛЬ, ТРИКОТАЖ)",0.652904,0.088174,0.034640,0.053055,0.601707,-0.022929,0.962914,-0.656012


In [171]:
all_clusters[group0]['FPMax'].sort_values('support',ascending=False).reset_index(drop=True)[0:10]

,support,itemsets
0,0.381386,"(ПОДГУЗНИКИ, ДЕТСКОЕ ПИТАНИЕ)"
1,0.129811,"(ПОДГУЗНИКИ, КОСМЕТИКА/ГИГИЕНА)"
2,0.084325,"(ДЕТСКОЕ ПИТАНИЕ, КОСМЕТИКА/ГИГИЕНА)"
3,0.073478,"(ТОВАРЫ ДЛЯ КОРМЛЕНИЯ, ДЕТСКОЕ ПИТАНИЕ)"
4,0.045836,"(ИГРУШКИ, ДЕТСКОЕ ПИТАНИЕ)"
5,0.034640,"(ТЕКСТИЛЬ, ТРИКОТАЖ, ДЕТСКОЕ ПИТАНИЕ)"
6,0.033940,"(ТОВАРЫ ДЛЯ КОРМЛЕНИЯ, КОСМЕТИКА/ГИГИЕНА)"
7,0.029391,"(ПОДГУЗНИКИ, ИГРУШКИ)"
8,0.022743,"(ПОДГУЗНИКИ, ТОВАРЫ ДЛЯ КОРМЛЕНИЯ)"
9,0.021693,"(ДЕТСКОЕ ПИТАНИЕ, КРУПНОГАБАРИТНЫЙ ТОВАР)"


In [172]:
all_clusters[group0]['Hmine'].sort_values('support',ascending=False).reset_index(drop=True)[0:10]

,support,itemsets
0,0.652904,(ДЕТСКОЕ ПИТАНИЕ)
1,0.592722,(ПОДГУЗНИКИ)
2,0.381386,"(ПОДГУЗНИКИ, ДЕТСКОЕ ПИТАНИЕ)"
3,0.289713,(КОСМЕТИКА/ГИГИЕНА)
4,0.151155,(ТОВАРЫ ДЛЯ КОРМЛЕНИЯ)
5,0.129811,"(ПОДГУЗНИКИ, КОСМЕТИКА/ГИГИЕНА)"
6,0.127362,(ИГРУШКИ)
7,0.088174,"(ТЕКСТИЛЬ, ТРИКОТАЖ)"
8,0.084325,"(ДЕТСКОЕ ПИТАНИЕ, КОСМЕТИКА/ГИГИЕНА)"
9,0.073478,"(ТОВАРЫ ДЛЯ КОРМЛЕНИЯ, ДЕТСКОЕ ПИТАНИЕ)"
